In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
DATA_FOLDER = 'data/'
WIKILITE_FOLDER = DATA_FOLDER + 'wiki_lite/'

In [3]:
train_df = pd.read_csv(DATA_FOLDER + 'train.csv')
test_df = pd.read_csv(DATA_FOLDER + 'test.csv')

enwinki_redirects = pd.read_csv(WIKILITE_FOLDER + 'enwiki_redirects.tsv', sep='\t')
item_aliases = pd.read_csv(WIKILITE_FOLDER + 'item_aliases.csv')
properties = pd.read_csv(WIKILITE_FOLDER + 'property.csv')
statements = pd.read_csv(WIKILITE_FOLDER + 'statements.csv')
wiki_items = pd.read_csv(WIKILITE_FOLDER + 'wiki_items.csv')

In [4]:
print('Redirects:')
display(enwinki_redirects)

print('Aliases:')
display(item_aliases)

print('Properties:')
display(properties)

print('Statements:')
display(statements)

print('Wiki items:')
display(wiki_items)

Redirects:


,!,Exclamation mark
0,! (CONFIG.SYS directive),CONFIG.SYS
1,! (Donnie Vie Album),Donnie Vie
2,! (Donnie Vie album),Donnie Vie
3,! (The Song Formerly Known As),Unit (album)
4,! (album),! (disambiguation)
...,...,...
8558897,󠁽,Tags (Unicode block)
8558898,󠁾,Tags (Unicode block)
8558899,󠁿,Tags (Unicode block)
8558900,􍁷,Private Use Areas


Aliases:


,item_id,en_alias
0,1,Our Universe
1,1,The Universe
2,1,The Cosmos
3,1,cosmos
4,2,Blue Planet
...,...,...
1495310,76858465,Photinus luciferin 4-monooxygenase (adenosine ...
1495311,76858465,firefly luciferase
1495312,76868858,12 Canum Venaticorum
1495313,76868858,Alpha Canum Venaticorum


Properties:


,property_id,en_label,en_description
0,6,head of government,"head of the executive power of this town, city..."
1,10,video,"relevant video. For images, use the property P..."
2,14,traffic sign,"graphic symbol describing the item, used at th..."
3,15,route map,image of route map at Wikimedia Commons
4,16,highway system,system (or specific country specific road type...
...,...,...,...
6980,7663,Scienza a due voci ID,identifier for an Italian female scientist in ...
6981,7665,FMV World ID,identifier of a video game in the FMV World da...
6982,7666,Visuotinė lietuvių enciklopedija ID,identifier for an item in the online edition o...
6983,7667,Hellenic Civil Aviation Authority airport code,NaN


Statements:


,source_item_id,edge_property_id,target_item_id
0,1,398,497745
1,1,398,1133705
2,1,398,1139177
3,1,398,7439451
4,1,398,15241043
...,...,...,...
26903183,77240068,31,16970
26903184,77240068,131,23556
26903185,77242291,17,145
26903186,77242291,131,23311


Wiki items:


,item_id,en_label,en_description,wikipedia_title
0,1,Universe,totality of space and all contents,Universe
1,2,Earth,third planet from the Sun in the Solar System,Earth
2,3,life,matter capable of extracting energy from the e...,Life
3,4,death,permanent cessation of vital functions,Death
4,5,human,"common name of Homo sapiens, unique extant spe...",Human
...,...,...,...,...
5216231,77042017,HR 4523,NaN,HD 102365
5216232,77043280,Charlie Johnston,NaN,Charlie Johnstone
5216233,77231860,Aldo Rossi,musician,Aldo Rossi (musician)
5216234,77240068,Ebenezer Baptist Church,"church in Atlanta, Georgia, USA",Ebenezer Baptist Church


In [5]:
train_df.head()

,id,token,entity_tag,full_mention,wiki_url
0,1,-DOCSTART- (1 EU),NaN,NaN,NaN
1,2,EU,B,EU,--NME--
2,3,rejects,NaN,NaN,NaN
3,4,German,B,German,http://en.wikipedia.org/wiki/Germany
4,5,call,NaN,NaN,NaN


In [6]:
entity_tokens = train_df[train_df['wiki_url'].notnull() & (train_df['wiki_url'] != '--NME--')]
entity_tokens['token'].unique()

array(['German', 'British', 'BRUSSELS', ..., 'Shinichi', 'Swe', 'Bradley'],
      dtype=object)

In [7]:
merged_wiki_items = wiki_items.merge(item_aliases, how='left', on='item_id')
merged_wiki_items.head()

,item_id,en_label,en_description,wikipedia_title,en_alias
0,1,Universe,totality of space and all contents,Universe,Our Universe
1,1,Universe,totality of space and all contents,Universe,The Universe
2,1,Universe,totality of space and all contents,Universe,The Cosmos
3,1,Universe,totality of space and all contents,Universe,cosmos
4,2,Earth,third planet from the Sun in the Solar System,Earth,Blue Planet


In [22]:
target_tokens = test_df[test_df['wiki_url'] == '?']
train_unique = target_tokens['token'].str.lower().unique()

In [26]:
test_unique = entity_tokens['token'].str.lower().unique()
test_unique

array(['german', 'british', 'brussels', ..., 'shinichi', 'swe', 'bradley'],
      dtype=object)

In [25]:
intersect = np.intersect1d(train_unique, test_unique)
intersect

array(['1860', 'abel', 'aberdeen', ..., 'zimbabwe', 'zinzan', 'zurich'],
      dtype=object)

In [12]:
# merged_statements = statements.copy(deep=True)
# merged_statements['source_item'] = merged_statements['source_item_id'].apply(lambda x: wiki_items[wiki_items['item_id'] == x]['wikipedia_title'])
# merged_statements['target_item'] = merged_statements['target_item_id'].apply(lambda x: wiki_items[wiki_items['item_id'] == x]['wikipedia_title'])
# merged_statements = merged_statements.merge(properties, how='left', left_on='edge_property_id', right_on='property_id')
# merged_statements.head()

KeyboardInterrupt: 